In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=f49737e569c7c0afc86793a15ca47557446a4264d5f304f5af7d7d48a7c57dc0
  Stored in directory: /root/.cache/pip/wheels/81/1b/c2/e41f4bddafe5564d11b2414a62dd7f5d75fcf65b2d7b7805c4
Successfully built pyspark


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pyspark #pyspark library

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import pyspark
from pyspark.sql import SparkSession

# Creating a SparkSession and giving an appName


In [4]:
spark = SparkSession.builder.appName("Spark_aggregate").getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/25 05:12:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/opt/conda/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


# List of Football Players, their nationality and Overall Rating 

In [5]:
data = {
    "Name": ["Cristiano", "Messi", "Mbappe","Benzema","Neymar","Bruno Fernandez","Pepe","Dybala","Griezman","Vinicius","Marcelo"],
    "Rating": [99, 98, 99,96,95,91,92,89,88,94,96],
    "Country": ["Portugal", "Argentina", "France","France","Brazil","Portugal","Portugal","Argentina","France","Brazil","Brazil"]
}
df_football = pd.DataFrame(data)

In [6]:

# creating a dataframe
df = spark.createDataFrame(data=df_football, schema = ["name","Rating","Country"])
df.printSchema()
df.show(truncate=False)

root
 |-- name: string (nullable = true)
 |-- Rating: long (nullable = true)
 |-- Country: string (nullable = true)



+---------------+------+---------+
|name           |Rating|Country  |
+---------------+------+---------+
|Cristiano      |99    |Portugal |
|Messi          |98    |Argentina|
|Mbappe         |99    |France   |
|Benzema        |96    |France   |
|Neymar         |95    |Brazil   |
|Bruno Fernandez|91    |Portugal |
|Pepe           |92    |Portugal |
|Dybala         |89    |Argentina|
|Griezman       |88    |France   |
|Vinicius       |94    |Brazil   |
|Marcelo        |96    |Brazil   |
+---------------+------+---------+



Casting Rating as Integer

In [7]:
from pyspark.sql.functions import col
df = df.withColumn("Rating",col("Rating").cast("integer"))

In [8]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- Rating: integer (nullable = true)
 |-- Country: string (nullable = true)



In [9]:
from pyspark.sql.functions import sum, col, desc

df.groupBy("Country") \
  .agg(sum("Rating").alias("sum_rating")) \
  .sort(desc("sum_rating")) \
  .show()

+---------+----------+
|  Country|sum_rating|
+---------+----------+
|   Brazil|       285|
|   France|       283|
| Portugal|       282|
|Argentina|       187|
+---------+----------+



# Count unique number of players from each country

In [10]:
from pyspark.sql.functions import col, countDistinct

In [11]:
df.groupBy("Country").agg(countDistinct("Rating").alias("Distinct_Players_Count")).sort(desc('Distinct_Players_Count')).show()

+---------+----------------------+
|  Country|Distinct_Players_Count|
+---------+----------------------+
|   France|                     3|
|   Brazil|                     3|
| Portugal|                     3|
|Argentina|                     2|
+---------+----------------------+



In [12]:
df_distinct = df.groupBy("Country").agg(countDistinct("Rating").alias("Distinct_Players_Count"))

In [13]:
df_distinct.show()

+---------+----------------------+
|  Country|Distinct_Players_Count|
+---------+----------------------+
|   France|                     3|
|Argentina|                     2|
|   Brazil|                     3|
| Portugal|                     3|
+---------+----------------------+



In [14]:
df.groupBy('Country').avg("Rating").alias("Average ratings by country").show()

+---------+-----------------+
|  Country|      avg(Rating)|
+---------+-----------------+
|Argentina|             93.5|
| Portugal|             94.0|
|   France|94.33333333333333|
|   Brazil|             95.0|
+---------+-----------------+



In [15]:
df.agg({'Rating':'Sum'}).show()

+-----------+
|sum(Rating)|
+-----------+
|       1037|
+-----------+

